# APIM ❤️ OpenAI

## 高度な負荷分散ラボ
![flow](../../images/advanced-load-balancing.gif)

カスタム[APIMポリシー](https://learn.microsoft.com/en-us/azure/api-management/api-management-howto-policies)に基づいた高度な負荷分散を試すためのプレイグラウンド。Azure OpenAIエンドポイントまたはモックサーバーのリストに対して負荷分散を行います。

このロードバランサーは、[スマートロードバランサーリポジトリ](https://github.com/andredewes/apim-aoai-smart-loadbalancing)からフォークしたカスタムポリシー構成に基づいており、以下の強化を追加しています：
- 名前付き値からロードバランサーの構成を読み込みます
- バックエンドを使用して、組み込みのサーキットブレーカ機能との組み合わせを可能にします
- エンドポイントの追加/変更やロードバランサーの構成のためにポリシーを変更する必要がありません
- 任意の数のOpenAIエンドポイントを動的にサポートします

### 目次
- [0️⃣ ノートブック変数の初期化](#0)
- [1️⃣ Azureリソースグループの作成](#1)
- [2️⃣ 🦾 Bicepを使用したデプロイの作成](#2)
- [3️⃣ デプロイの出力を取得](#3)
- [🧪 直接HTTPコールを使用したAPIのテスト](#requests)
- [🧪 Azure OpenAI Python SDKを使用したAPIのテスト](#sdk)
- [🗑️ リソースのクリーンアップ](#clean)

### バックログ
- 重み付きバックエンドプール
- 状態を持つ[アシスタントAPI](https://cookbook.openai.com/examples/assistants_api_overview_python)のスティッキネス

### 前提条件
- [Python 3.8以降のバージョン](https://www.python.org/)がインストールされていること
- [VS Code](https://code.visualstudio.com/)がインストールされ、[Jupyterノートブック拡張機能](https://marketplace.visualstudio.com/items?itemName=ms-toolsai.jupyter)が有効になっていること
- [Azure CLI](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)がインストールされていること
- Contributor権限を持つ[Azureサブスクリプション](https://azure.microsoft.com/en-us/free/)があること
- [Azure OpenAIへのアクセス](https://aka.ms/oai/access)が許可されていること、またはモックサービスを有効にすること
- [Azure CLIでAzureにサインイン](https://learn.microsoft.com/en-us/cli/azure/authenticate-azure-cli-interactively)していること

<a id='0'></a>
### 0️⃣ ノートブック変数の初期化

- リソースはサブスクリプションIDに基づいた一意の文字列でサフィックスされます
- ```mock_webapps``` 変数は、モック機能のためにデプロイされたWebアプリのリストを設定します。モックサービスでOpenAIの動作をシミュレートするために、```openai_resources``` リストをクリアします。
- ロケーションパラメータは、[Azureリージョンごとの製品の利用可能性](https://azure.microsoft.com/en-us/explore/global-infrastructure/products-by-region/?cdn=disable&products=cognitive-services,api-management)に基づいて調整してください。
- OpenAIモデルとバージョンは、[リージョンごとの利用可能性](https://learn.microsoft.com/en-us/azure/ai-services/openai/concepts/models)に基づいて調整してください。

In [ ]:
import os
import json
import datetime
import requests

deployment_name = os.path.basename(os.path.dirname(globals()['__vsc_ipynb_file__']))
resource_group_name = f"lab-{deployment_name}" # change the name to match your naming style
resource_group_location = "westeurope"
apim_resource_name = "apim31"
apim_resource_location = "westeurope"
apim_resource_sku = "Basicv2"
openai_resources = [ {"name": "openai1", "location": "swedencentral"}, {"name": "openai2", "location": "francecentral"}  ] # list of OpenAI resources to deploy. Clear this list to use only the mock resources
openai_lb_config = [ {"name": "openai1", "priority": 1, "weight": 100}, {"name": "openai2", "priority": 2, "weight": 300}  ] # advanced load balancer configuration for OpenAI resources that will be stored as a named value
openai_resources_sku = "S0"
openai_model_name = "gpt-35-turbo"
openai_model_version = "0613"
openai_deployment_name = "gpt-35-turbo"
openai_api_version = "2024-02-01"
openai_specification_url='https://raw.githubusercontent.com/Azure/azure-rest-api-specs/main/specification/cognitiveservices/data-plane/AzureOpenAI/inference/stable/' + openai_api_version + '/inference.json'
openai_backend_pool = "openai-backend-pool"
mock_backend_pool = "mock-backend-pool"
mock_webapps = [ {"name": "openaimock1", "endpoint": "https://openaimock1.azurewebsites.net"}, {"name": "openaimock2", "endpoint": "https://openaimock2.azurewebsites.net"} ]
mock_lb_config = [ {"name": "openaimock1", "priority": 1, "weight": 100}, {"name": "openaimock2", "priority": 2, "weight": 300}  ] # advanced load balancer configuration for mock services that will be stored as a named value


<a id='1'></a>
### 1️⃣ Azureリソースグループの作成
このラボでデプロイされるすべてのリソースは、指定されたリソースグループに作成されます。既存のリソースグループを使用する場合は、このステップをスキップしてください。

In [ ]:
resource_group_stdout = ! az group create --name {resource_group_name} --location {resource_group_location}
if resource_group_stdout.n.startswith("ERROR"):
    print(resource_group_stdout)
else:
    print("✅ Azure Resource Group ", resource_group_name, " created ⌚ ", datetime.datetime.now().time())

<a id='2'></a>
### 2️⃣ 🦾 Bicepを使用したデプロイの作成

このラボでは、[Bicep](https://learn.microsoft.com/en-us/azure/azure-resource-manager/bicep/overview?tabs=bicep)を使用して、デプロイされるすべてのリソースを宣言的に定義します。異なる構成を試すために、パラメータや[main.bicep](main.bicep)を直接変更してください。

In [ ]:

loadbalancer_config = openai_lb_config if len(openai_resources) > 0 else mock_lb_config
loadbalancer_config = json.dumps(loadbalancer_config).replace("\"", "'")

bicep_parameters = {
  "$schema": "https://schema.management.azure.com/schemas/2019-04-01/deploymentParameters.json#",
  "contentVersion": "1.0.0.0",
  "parameters": {
    "mockWebApps": { "value": mock_webapps },
    "mockBackendPoolName": { "value": mock_backend_pool },
    "openAIBackendPoolName": { "value": openai_backend_pool },
    "openAIConfig": { "value": openai_resources },
    "openAILoadBalancingConfigValue": { "value": loadbalancer_config },
    "openAIDeploymentName": { "value": openai_deployment_name },
    "openAISku": { "value": openai_resources_sku },
    "openAIModelName": { "value": openai_model_name },
    "openAIModelVersion": { "value": openai_model_version },
    "openAIAPISpecURL": { "value": openai_specification_url },
    "apimResourceName": { "value": apim_resource_name},
    "apimResourceLocation": { "value": apim_resource_location},
    "apimSku": { "value": apim_resource_sku}
  }
}
with open('params.json', 'w') as bicep_parameters_file:
    bicep_parameters_file.write(json.dumps(bicep_parameters))

! az deployment group create --name {deployment_name} --resource-group {resource_group_name} --template-file "main.bicep" --parameters "params.json"

<a id='3'></a>
### 3️⃣ デプロイの出力を取得

テストの準備が整う前に、ゲートウェイURLとサブスクリプションを取得する必要があります。

In [ ]:
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimSubscriptionKey.value -o tsv
apim_subscription_key = deployment_stdout.n
deployment_stdout = ! az deployment group show --name {deployment_name} -g {resource_group_name} --query properties.outputs.apimResourceGatewayURL.value -o tsv
apim_resource_gateway_url = deployment_stdout.n
print("👉🏻 API Gateway URL: ", apim_resource_gateway_url)

<a id='requests'></a>
### 🧪 直接HTTPコールを使用したAPIのテスト
Requestsは、ここで生のAPIリクエストを行い、レスポンスを検査するために使用される、エレガントでシンプルなPython用のHTTPライブラリです。

In [ ]:
url = apim_resource_gateway_url + "/openai/deployments/" + openai_deployment_name + "/chat/completions?api-version=" + openai_api_version
if len(openai_resources) > 0:
    messages={"messages":[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]}
elif len(mock_webapps) > 0:
    messages={
        "messages": [
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0},
                        "openaimock1.azurewebsites.net": {"response_status_code": 429}
                    }
                }
            }
        ]
    }
response = requests.post(url, headers = {'api-key':apim_subscription_key}, json = messages)
print("status code: ", response.status_code)
print("headers ", response.headers)
print("x-ms-region: ", response.headers.get("x-ms-region")) # this header is useful to determine the region of the backend that served the request
print("x-ms-openai: ", response.headers.get("x-ms-openai")) 

if (response.status_code == 200):
    data = json.loads(response.text)
    print("response: ", data.get("choices")[0].get("message").get("content"))
else:
    print(response.text)

<a id='sdk'></a>
### 🧪 Azure OpenAI Python SDKを使用したAPIのテスト
OpenAPIは広く使用されている[Pythonライブラリ](https://github.com/openai/openai-python)を提供しています。このライブラリには、すべてのリクエストパラメータとレスポンスフィールドの型定義が含まれています。このテストの目的は、APIMがOpenAIへのリクエストをシームレスにプロキシし、その機能を妨げることなく動作することを確認することです。
- 注意: このステップを実行する前に、ターミナルで```pip install openai```を実行してください。

In [ ]:
from openai import AzureOpenAI
if len(openai_resources) > 0:
    messages=[
        {"role": "system", "content": "You are a sarcastic unhelpful assistant."},
        {"role": "user", "content": "Can you tell me the time, please?"}
    ]
elif len(mock_webapps) > 0:
    messages=[
            {
                "role": "system", 
                "content": {
                    "simulation": {
                        "default": {"response_status_code": 200, "wait_time_ms": 0},
                        "openaimock1.azurewebsites.net": {"response_status_code": 429}
                    }
                }
            }
        ]
client = AzureOpenAI(
    azure_endpoint=apim_resource_gateway_url,
    api_key=apim_subscription_key,
    api_version=openai_api_version
)
response = client.chat.completions.create(model=openai_model_name, messages=messages)
print(response.choices[0].message.content)

<a id='clean'></a>
### 🗑️ リソースのクリーンアップ

ラボが終了したら、追加の料金を避け、Azureサブスクリプションを整理するために、デプロイしたすべてのリソースをAzureから削除する必要があります。
そのためには、[リソースのクリーンアップノートブック](clean-up-resources.ipynb)を使用してください。